# Original Problem Formulation
The plugin implements a double pendulum internally. Double pendulums are classic
examples of chaotic systems, and notably, the differential equations describing
their motion, cannot be solved symbolically.

Typically, a double-pendulum implementation uses algorithms such as Runge-Kutta
to improve the quality of the numeric approximation.

The double-pendulum equations are the following:

$\dot{\theta_1}(t)=\omega_1$ <br/>
$\dot{\theta_2}(t)=\omega_2$ <br/>
$\dot{\omega_1}(t)=\dfrac
{-g(2m_1+m_2)\sin(\theta_1)-m_2g\sin(\theta_1-2\theta_2)-2\sin(\theta_1-\theta_2)m_2(\omega_2^2L_2+\omega_1^2L_1\cos(\theta_1-\theta_2))}
{L_1(2m_1 + m_2 - m_2\cos(2\theta_1-2\theta_2))}$ <br/>
$\dot{\omega_2}(t)=\dfrac
{2\sin(\theta_1-\theta_2)(\omega_1^2L_1(m_1+m_2)+g(m_1+m_2)\cos\theta_1+\omega_2^2L_2m_2\cos(\theta_1-\theta_2))}
{L_2(2m_1+m_2-m_2\cos(2\theta_1-2\theta_2))}$ <br/>

The numeric solution would then look like this: <br/>
$\theta_1(0) = $ initial value set by user. <br/>
$\theta_2(0) = $ initial value set by user. <br/>
$\omega_1(0) = $ initial value set by user. <br/>
$\omega_2(0) = $ initial value set by user. <br/>
$\theta_1(t) = \theta_1(t-\Delta t) + RK4(\dot{\theta_1}, \Delta t)$ Runge-kutta, 
giving the function pointer for calulating $\Delta\theta$<br/>
$\theta_2(t) = \theta_2(t - \Delta t) + RK4(\dot{\theta_2}, \Delta t)$ <br/>
$\omega_1(t) = \omega_1(t - \Delta t) + RK4(\dot{\omega_1}, \Delta t)$ <br/>
$\omega_2(t) = \omega_2(t - \Delta t) + RK4(\dot{\omega_2}, \Delta t)$ <br/>

Where the $RK4$ differential step is defined as follows: <br/>
$\dot{y} = f(t, y)$ <br/>
$y(n+1) = y(n) + RK4(f, t)$ <br/>
$RK4(f, t) = \Delta t\sum b_i k_i$ <br/>
$k_1 = f(t_n, y_n)$ <br/>
$k_2 = f(t_n + \dfrac{1}{3}\Delta t, y_n + \Delta t(\dfrac{1}{3}k_1))$ <br/>
$k_3 = f(t_n + \dfrac{2}{3}\Delta t, y_n + \Delta t(-\dfrac{1}{3}k_1 + k_2))$ <br/>
$k_4 = f(t_n + \Delta t, y_n + \Delta t(k_1 - k_2 + k_3))$ <br/>
$b_i \in \left\{\dfrac{1}{8}, \dfrac{3}{8}, \dfrac{3}{8}, \dfrac{1}{8}\right\}$

# Our Implementation
Our implementation takes a few liberties, specifically, the two angular velocities are capped by a $tanh()$ saturation function.
The angular acceleration is capped by an amount dependant on the frequency (?) of the system. 

Additional, the angular velocity is "crossfaded" with a constant that can be set by the user. The constant is scaled such that crossfading 100% towards the constant will produce a perfect oscillation at a frequency specified by the user.

$const = \dfrac{\tau}{SR/f}$, where $\tau = 2\pi$

$XFade(\omega, const, fade) = (const\cdot fade) + (\omega\cdot(1-fade))$

Finally, the outputs of the plugin are: <br/>
$L = \sin(\theta_1)$ <br/>
$R = \sin(\theta_2)$ <br/>


# Optimization Considerations
To avoid computing the same values multuple times, the equations have been split up into multuple symbols, so that they can be reused.

````py
gamma =       th_1 - th_2
M =           m_1 + m_2
w1_2 =        w1*w1
w2_2 =        w2*w2
2_sin_gamma = 2*sin(gamma)
cos_gamma =   cos(gamma)
aux_0 =       2*m1 + m2 - m2*cos(2*gamma)
aux_1 =       -g*(2*m1 + m2)*sin(th_1) - m2*g*sin(th_1 - 2*th_2)
aux_2 =       2_sin_gamma*m2*(w2_2*L1 + w1_2*L1*cos_gamma)
aux_3 =       2_sin_gamma*(w1_2*L1*M + g*M*cos(th_1) + w2_2*L2*m2*cos_gamma)
dw1 =         (aux_1 - aux_2)/(L1*aux_0)
dw2 =         aux_3/(L2*aux_0)
````

Runge-kutta will be quite expensive, as the function above has to be called many times. Use criterion optimization library.